## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [3]:
bike_df = pd.read_csv('bikeshare.csv')

In [4]:
len(bike_df.index)

10886

In [5]:
len(bike_df.columns)

12

In [6]:
bike_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [7]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [8]:
bike_df['datetime'] = pd.to_datetime(bike_df.datetime)
bike_df['datetime']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: datetime, Length: 10886, dtype: datetime64[ns]

In [9]:
bike_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

In [10]:
'''
bike_df['year'] = bike_df['datetime'].apply(lambda _: datetime.strptime(_,"%Y-%m-%d %H:%M:%S" ).year)
bike_df['month'] = bike_df['datetime'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))
bike_df['day'] = bike_df['datetime'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))
bike_df['time'] = bike_df['datetime'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))
'''

'\nbike_df[\'year\'] = bike_df[\'datetime\'].apply(lambda _: datetime.strptime(_,"%Y-%m-%d %H:%M:%S" ).year)\nbike_df[\'month\'] = bike_df[\'datetime\'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))\nbike_df[\'day\'] = bike_df[\'datetime\'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))\nbike_df[\'time\'] = bike_df[\'datetime\'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))\n'

In [11]:
#apply() 함수와 lambda 를 이용해서 만드는 법을 모르겠어서 일단 dt로 진행했습니다!
import datetime as dt
bike_df['year'] = bike_df['datetime'].dt.year
bike_df['month'] = bike_df['datetime'].dt.month
bike_df['day'] = bike_df['datetime'].dt.day
bike_df['hour'] = bike_df['datetime'].dt.hour

In [12]:
bike_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [13]:
bike_df.sample(n=3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
2106,2011-05-14 16:00:00,2,0,0,2,22.14,25.760,94,12.9980,107,188,295,2011,5,14,16
2008,2011-05-10 14:00:00,2,0,1,2,25.42,31.060,33,0.0000,42,134,176,2011,5,10,14
3230,2011-08-04 12:00:00,3,0,1,2,31.16,35.605,62,15.0013,53,152,205,2011,8,4,12


* datatime, causal, registered 칼럼을 삭제 하시오.(casual + registered = count이므로 따로 필요하지 않습니다.) 

In [14]:
bike_df.drop(["datetime", "casual", "registered"], axis=1, inplace=True)
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4


## 2. 선형 회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. (종속변수 = count)

In [15]:
#종속변수는 Survived
X_fearues = bike_df.drop('count', axis = 1)
y_target = bike_df['count']

### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [16]:
from sklearn.preprocessing import OneHotEncoder 

cat_encoder = OneHotEncoder()
bike_df_cat_1hot = cat_encoder.fit_transform( bike_df[["year", "month", "day", "hour", "holiday", "workingday", "season", "weather"]])
bike_df_cat_1hot

X_features_ohe = pd.get_dummies(bike_df, columns = ["year", "month", "day", "hour", "holiday", "workingday", "season", "weather"], drop_first=True)

In [17]:
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4


### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_fearues, y_target, test_size=0.3, random_state=0)

### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

lin_reg.predict(X_test)

array([320.78637463, 251.1873687 , 241.70971289, ..., 315.23151252,
       181.92737435, 166.29021343])

In [20]:
pred = lin_reg.predict(X_test)

In [29]:
def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

print('RMSLE:', rmse(y_test,pred))

RMSLE: 140.89958867165507


#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [31]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(penalty="None", max_iter=1000, tol=1e-3, eta0=0.1)
sgd_reg.fit(X_train, y_train)
sgd_reg.predict(X_test)

array([-5.69683290e+15, -2.70585051e+15, -6.07181915e+15, ...,
       -3.71849230e+15, -4.63463520e+15, -6.34106720e+15])

In [32]:
pred = sgd_reg.predict(X_test)

In [33]:
def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

print('RMSLE:', rmse(y_test,pred))

RMSLE: 5442298548088318.0


#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [35]:
from sklearn.linear_model import Ridge, Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_train, y_train)
lasso_reg.predict(X_test)

pred_lasso = lasso_reg.predict(X_test)

ridge_reg = Ridge(alpha=0.1)
ridge_reg.fit(X_train, y_train)
ridge_reg.predict(X_test)
ridge_reg.intercept_, ridge_reg.coef_
pred_ridge = ridge_reg.predict(X_test)

In [36]:
def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

print('lasso_RMSLE:', rmse(y_test,pred_lasso))
print('lasso_intercept:', lasso_reg.intercept_)
print('lasso_coefficient:', lasso_reg.coef_)

lasso_RMSLE: 140.89862972310226
lasso_intercept: -163182.58916639184
lasso_coefficient: [-6.35905308 -2.77794117  2.93199509 -5.2006848   2.66127106  3.75303067
 -2.06192368  0.31933133 81.14183089  9.71108029  0.47167525  7.89917515]


In [37]:
print('ridge_RMSLE:', rmse(y_test,pred_ridge))
print('ridge_intercept:', ridge_reg.intercept_)
print('ridge_coefficient:', ridge_reg.coef_)

ridge_RMSLE: 140.89961265573186
ridge_intercept: -164043.8140488084
ridge_coefficient: [-7.99598527 -5.90391576  3.09939923 -5.53979213  2.579876    3.82709881
 -2.05630338  0.32831974 81.57023226 10.23336989  0.47379729  7.90417675]


## 3. 로지스틱 회귀 분석 실습 _ 와인 종류 분류하기 : 와인의 화학 조성을 사용하여 와인의 종류를 예측하기 위한 데이터이다. 

In [38]:
# 와인 데이터를 불러오기 위한 코드입니다. 꼭 실행해주세요.
from sklearn import datasets

ㅇ 타겟 데이터 : 와인의 종류 0, 1, 2의 세가지 값

ㅇ 특징 데이터

* 알콜(Alcohol)

* 말산(Malic acid)

* 회분(Ash)

* 회분의 알칼리도(Alcalinity of ash)

* 마그네슘(Magnesium)

* 총 폴리페놀(Total phenols)

* 플라보노이드 폴리페놀(Flavanoids)

* 비 플라보노이드 폴리페놀(Nonflavanoid phenols)

* 프로안토시아닌(Proanthocyanins)

* 색상의 강도(Color intensity)

* 색상(Hue)

* 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)

* 프롤린(Proline)

####3-1. 와인 데이터 가져오기 

In [39]:
wine_data = datasets.load_wine()

더 자세한 데이터에 대한 설명을 보고싶으시다면 아래의 코드를 실행해주시면 됩니다. 

In [40]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

#### 3-2. 입력변수와 목표변수 설정하기
* 입력변수를 사용하기 편하게 pandas dataframe으로 변환해주세요.
  - 입력변수는 data (.data) 함수를 써서 가져오시면 됩니다.
  - 칼럼명 가져오기 : .feature_names 
* 목표변수를 사용하기 편하게 pandas dataframe으로 변환해주세요
  - 목표변수는 target (.target) 함수를 써서 가져요오시면 됩니다. 
  - 칼럼명은 'target'으로 지정해주세요

In [42]:
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

y = pd.DataFrame(wine_data.target, columns=['target'])

#### 3-3. 훈련 데이터, 테스트 데이터 만들기
* train_test_split 함수를 사용해 테스트 셋의 비율이 0.2가 되도록 X_train, X_test, y_train, y_test 을 만들어주세요. random_state=5로 설정해주세요.
* 경고메세지를 없애도록 y_train = y_train.values.ravel() 를 실행해주세요

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
y_train = y_train.values.ravel()

#### 3-4. 모델훈련
* 로지스틱회귀 함수에서 옵션을 solver='saga', max_iter=7500으로 설정해주세요.
* 학습데이터로 모델을 학습시켜주세요

In [44]:
# 필요한 라이브러리 import
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='saga', max_iter=7500)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=7500, solver='saga')

#### 3-5. 예측
* 테스트 데이터 셋으로 로지스틱 회귀 모델을 이용해 와인 데이터 분류를 예측해 결과를 y_test_predict 변수에 담아주세요
* score함수를 사용해 로지스틱 회귀 모델의 성능을 score 변수에 담아주세요
* y_test_predict와 score 결과를 확인해주세요.

In [45]:
# 로지스틱 회귀 모델를 이용해서 각 와인 데이터 분류를 예측함
y_test_predict = log_reg.predict(X_test)

# 로지스틱 회귀 모델의 성능 확인 (정확성 %를 리턴함)
score = log_reg.score(X_test, y_test)

# 결과확인 
y_test_predict, score

(array([0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2]),
 0.75)

#### (선택) 3-6. GridSearchCV 복습

`LogisticRegression` 모델에 `param_grid`를 사용하여 GridSearchCV를 진행해주세요.
- cv = 5
- 추가적으로 코드를 작성하실 필요 없이, 골뱅이@@@ 부분의 코드를 완성해서 출력해주시면 됩니다. 

In [46]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

# param_grid (수정하셔도 됩니다)
params = {
    'penalty':['l1', 'l2', 'elasticnet'],        # l1 is Lasso, l2 is Ridge
    'C': np.linspace(0.00002,1,100)
}

# GridSearch
grid_search = GridSearchCV(log_reg, param_grid=params, cv = 5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=7500, solver='saga'),
             param_grid={'C': array([2.00000000e-05, 1.01208081e-02, 2.02216162e-02, 3.03224242e-02,
       4.04232323e-02, 5.05240404e-02, 6.06248485e-02, 7.07256566e-02,
       8.08264646e-02, 9.09272727e-02, 1.01028081e-01, 1.11128889e-01,
       1.21229697e-01, 1.31330505e-01, 1.41431313e-01, 1.51532121e-01,
       1.61632929e...
       7.67681414e-01, 7.77782222e-01, 7.87883030e-01, 7.97983838e-01,
       8.08084646e-01, 8.18185455e-01, 8.28286263e-01, 8.38387071e-01,
       8.48487879e-01, 8.58588687e-01, 8.68689495e-01, 8.78790303e-01,
       8.88891111e-01, 8.98991919e-01, 9.09092727e-01, 9.19193535e-01,
       9.29294343e-01, 9.39395152e-01, 9.49495960e-01, 9.59596768e-01,
       9.69697576e-01, 9.79798384e-01, 9.89899192e-01, 1.00000000e+00]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

학습된 gridsearch 모델에 저장되어 있는 best parameter, best score 를 출력해주세요.

In [47]:
grid_search.best_params_, grid_search.best_score_

({'C': 0.08082646464646465, 'penalty': 'l2'}, 0.9302955665024631)